In [15]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any
from ingestion.db_utils import load_report_params
from pathlib import Path

db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"


conn = sqlite3.connect(db_path)
cur = conn.cursor()
db_path = Path(conn.execute("PRAGMA database_list").fetchone()[2])
report = 'Quarterly_Report'
report_params = load_report_params(report_name=report, db_path=db_path)
report_quarter = report_params.get("overviewDate")

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

df = fetch_latest_table_data(conn, "c0_budgetary_execution_details", cutoff)


df.head(2)


,Financial Management Area,FMA Desc,Fiscal Year,Administrative/Operational Flg,Budget Address,Official Budget Line,Fund Source,Fund Source Desc,Budget Period,Commitment Item,...,Commitment Appropriation,Committed Amount,Commitment Available,%,Payment Appropriation,Paid Amount,Payment Available,%.1,upload_id,uploaded_at
0,ECOM,European Commission,2025,OPERA,EARN/N-2024-ERCEA<RTD-E.01020101-HORIZONEU_21_...,01 02 01 01,EARN/N,Ext. Assigned Rev. - General Rule,2024,E.01020101,...,94097760.18,0.0,94097760.18,0.0,7.918999e+08,20608882.37,7.712910e+08,0.026025,6,2025-05-05T08:46:12.088090
1,ECOM,European Commission,2025,OPERA,EARN/N-2024-ERCEA<RTD-E.01020101-HORIZONEU_21_...,01 02 01 01,EARN/N,Ext. Assigned Rev. - General Rule,2024,E.01020101,...,2455309.68,0.0,2455309.68,0.0,0.000000e+00,0.00,0.000000e+00,0.000000,6,2025-05-05T08:46:12.088090


In [19]:

# Filter for current year and specific Fund Sources
df = df[df["Budget Period"] == current_year].copy()
logging.debug(f"After filtering Budget Period == {current_year}: {df.shape[0]} rows")

df = df[df["Fund Source"].isin(["VOBU", "EFTA"])].copy()
logging.debug(f"After filtering Fund Source in ['VOBU', 'EFTA']: {df.shape[0]} rows")

# Map Functional Area to Programme
df.loc[:, "Programme"] = df["Functional Area Desc"].replace({
    "HORIZONEU_21_27": "HE",
    "H2020_14_20": "H2020"
})

# Filter for HE programme
df = df.loc[df["Programme"] == "HE"].copy()
logging.debug(f"After filtering Programme == 'HE': {df.shape[0]} rows")


# Budget-type mapping helper
def map_budget_type(val):
    if pd.isna(val):
        return None
    v = str(val).upper()
    if "EMPTY" in v:
        return "Main Calls"
    if "EXPERTS" in v:
        return "Experts"
    return val

df.loc[:, "Budget_Address_Type"] = df["Budget Address"].apply(map_budget_type)
logging.debug(f"Unique Budget_Address_Type values: {df['Budget_Address_Type'].unique()}")

# Fund-type mapping helper
def map_fund_type(val):
    if pd.isna(val):
        return None
    v = str(val).upper()
    if "VOBU" in v:
        return "VOBU/EFTA"
    if "EFTA" in v:
        return "VOBU/EFTA"
    if "IAR2/2" in v:
        return "VOBU/EFTA"
    return val

df['Fund Source'] = df['Fund Source'].apply(map_fund_type)
logging.debug(f"Unique Fund Source values after mapping: {df['Fund Source'].unique()}")

# Group by Fund Source and Budget_Address_Type
agg = df.groupby(["Fund Source", "Budget_Address_Type"])[
    ["Commitment Appropriation", "Committed Amount", "Commitment Available "]
].sum().reset_index()
logging.debug(f"Aggregated DataFrame shape: {agg.shape}")

# Compute percentage
agg["%"] = agg["Committed Amount"] / agg["Commitment Appropriation"].replace(0, pd.NA)
agg["%"] = agg["%"].fillna(0)  # Handle division by zero

# Rename columns
agg = agg.rename(columns={
    "Commitment Appropriation": "Available_Commitment_Appropriations",
    "Committed Amount": "L1_Commitment",
    "Commitment Available ": "RAC_on_Appropriation",
    "%": "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations",
    "Budget_Address_Type": "Budget Address Type"
})

total_row = pd.DataFrame({
    "Fund Source": ["VOBU/EFTA"],
    "Budget Address Type": ["Total"],
    "Available_Commitment_Appropriations": [agg["Available_Commitment_Appropriations"].sum()],
    "L1_Commitment": [agg["L1_Commitment"].sum()],
    "RAC_on_Appropriation": [agg["RAC_on_Appropriation"].sum()],
    "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations": [
        agg["L1_Commitment"].sum() / agg["Available_Commitment_Appropriations"].sum()
        if agg["Available_Commitment_Appropriations"].sum() != 0 else 0
    ]
})
agg = pd.concat([agg, total_row], ignore_index=True)

agg

,Fund Source,Budget Address Type,Available_Commitment_Appropriations,L1_Commitment,RAC_on_Appropriation,ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations
0,VOBU/EFTA,Experts,2.000000e+07,17040224.92,2.959775e+06,0.852011
1,VOBU/EFTA,Main Calls,2.198255e+09,0.00,2.198255e+09,0.000000
2,VOBU/EFTA,Total,2.218255e+09,17040224.92,2.201215e+09,0.007682


In [21]:
tot = agg.iloc[-1]         # last row *is* the total

ratio_pct = f"{tot['ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations']*100:.2f}%"
avail_bln = f"{tot['Available_Commitment_Appropriations']/1e9:.2f}"

budget_impl = (
    "Consumption of commitment appropriations out of the total "
    "available credits (C1/E0-HEU)"
)
overview     = f"{ratio_pct} (out of € {avail_bln} bln)"

summary = pd.DataFrame(
    {   
        'Indicator' : "Budget implementation",
        "Indicator_Description": [budget_impl],
        f'{report_quarter}': [overview],
        'Target' : '100%'
    }
)

summary

,Indicator,Indicator_Description,March 2025,Target
0,Budget implementation,Consumption of commitment appropriations out o...,0.77% (out of € 2.22 bln),100%
